# Testing with nbdev

> Highlighting some key ideas of nbdev

One of the nice features that can be useful with nbdev is the ability to add tests inside of the notebooks. This occurs as just another cell and I will demonstrate it below

In [ ]:
#| default_exp agent

In [ ]:
#| export
import datetime
import os
import openai
from fastcore.basics import store_attr
from fastcore.test import test_eq

In [ ]:
#| export 
openai.api_key = os.environ['OPENAI_API_KEY']
model_engine = "gpt-3.5-turbo"

In [ ]:
#| export
from enum import Enum

In [ ]:
#| export
class Role(Enum):
    "This corresponds to the roles that openai allows for the ChatGPT API"
    SYSTEM = "system"
    USER = "user"
    ASSISTANT = "assistant"
    FUNCTION = "function"

In [ ]:
#| echo: false
for r in Role:
    print(f"{r}: \"{r.value}\"")

Role.SYSTEM: "system"
Role.USER: "user"
Role.ASSISTANT: "assistant"
Role.FUNCTION: "function"


In [ ]:
#| export
def create_message(role_type:Role, # Whether this message is the system, user, or assistant talking
                   content:str # A string that can be used 
                  ):
    return {"role":role_type.value, "content": content}

In [ ]:
messages = []

In [ ]:
messages.append(create_message(Role.SYSTEM, "You are the organizer of a cool meetup about python. You are trying to get the speaker to wrap up his talk"))

In [ ]:
messages.append(create_message(Role.USER, "And now, I will show you my 19th example on why nbdev is cool"))

In [ ]:
# Generate text
completion = openai.ChatCompletion.create(
    model=model_engine,
    messages=messages,
)

In [ ]:
acompletion = openai.ChatCompletion.acreate(
    model=model_engine,
    messages=messages,
)

In [ ]:
completion_type2 = await acompletion

In [ ]:
completion.to_dict_recursive()

{'id': 'chatcmpl-8HmXYSh0qW6hsD9Br6z28voGbiGgp',
 'object': 'chat.completion',
 'created': 1699248856,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Thank you for sharing your 19th example! It's been really interesting and informative. However, we are running a bit short on time, and we have a few things left to cover. So, if you could kindly wrap up your talk and maybe highlight the main takeaways from your examples, that would be great. We want to make sure we have enough time for questions and discussion at the end. Thank you!"},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 52, 'completion_tokens': 85, 'total_tokens': 137}}

In [ ]:
completion.choices

[<OpenAIObject> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "Thank you for sharing your 19th example! It's been really interesting and informative. However, we are running a bit short on time, and we have a few things left to cover. So, if you could kindly wrap up your talk and maybe highlight the main takeaways from your examples, that would be great. We want to make sure we have enough time for questions and discussion at the end. Thank you!"
   },
   "finish_reason": "stop"
 }]

In [ ]:
completion['choices']

[<OpenAIObject> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "Thank you for sharing your 19th example! It's been really interesting and informative. However, we are running a bit short on time, and we have a few things left to cover. So, if you could kindly wrap up your talk and maybe highlight the main takeaways from your examples, that would be great. We want to make sure we have enough time for questions and discussion at the end. Thank you!"
   },
   "finish_reason": "stop"
 }]

In [ ]:
test_eq(completion['choices'], completion.choices)

In [ ]:
type(completion)

openai.openai_object.OpenAIObject

In [ ]:
#|output: asis
print(completion.choices[0].message.content)

Thank you for sharing your 19th example! It's been really interesting and informative. However, we are running a bit short on time, and we have a few things left to cover. So, if you could kindly wrap up your talk and maybe highlight the main takeaways from your examples, that would be great. We want to make sure we have enough time for questions and discussion at the end. Thank you!


In [ ]:
#| export
class Speaker:
    "A speaker is somebody that will talk about things!"
    def __init__(self, name:str, backstory:str=None, mannerisms:str=None, relationships:dict=None, model_engine="gpt-3.5-turbo"): # Test
        store_attr()
        self.messages=[]
        self.setup_backstory()
        self.setup_mannerisms()
        self.setup_universe()

    def setup_universe(self):
        # self.messages.append(create_message(Role.SYSTEM, f'you are one of a handful of members in an improv group in front of a live audience.'))
        self.messages.append(create_message(Role.SYSTEM, f'Your message should be no more than a paragraph'))
        self.messages.append(create_message(Role.SYSTEM, f'Responses should like like this YOUR_NAME: Response to the previous messages.'))
        self.messages.append(create_message(Role.SYSTEM, f'If needed you can do actions by putting them in asterisks'))
        self.messages.append(create_message(Role.SYSTEM, f'Use markdown to make the output look pretty'))
        self.messages.append(create_message(Role.SYSTEM, f'You can only write your own viewpoint of the story. Never write the other persons response'))
    
    def setup_backstory(self):
        if self.backstory is None:
            self.messages.append(create_message(Role.SYSTEM, f'Your name is {self.name}. Choose a random backstory and make sure to tell me the backstory at the top of the next message'))
        else:
            self.messages.append(create_message(Role.SYSTEM, f'Your name is {self.name} and here is your backstory: {self.backstory}'))

    def setup_mannerisms(self):
        if self.mannerisms is None:
            self.messages.append(create_message(Role.SYSTEM, f'Choose some random mannerisms and make sure to tell me what the mannerisms are at the top of the next message'))
        else:
            self.messages.append(create_message(Role.SYSTEM, f'Here are some of your mannerisms: {self.mannerisms}'))

    def setup_scene(self, scene:str):
        self.messages.append(create_message(Role.SYSTEM, scene))
        return "**SCENE PLOT: "+scene+"**"
    
    def listen_to_input(self,inp):
        self.messages.append(create_message(Role.USER, inp))

    def talk(self, max_tokens:int=300):
        completion = openai.ChatCompletion.create(
            model=self.model_engine,
            messages=self.messages,
            max_tokens=max_tokens
        )
        self.messages.append(create_message(Role.ASSISTANT, completion.choices[0].message.content))
        return completion.choices[0].message.content

In [ ]:
speaker = Speaker('Kevin')

In [ ]:
speaker.messages

[{'role': 'system',
  'content': 'Your name is Kevin. Choose a random backstory and make sure to tell me the backstory at the top of the next message'},
 {'role': 'system',
  'content': 'Choose some random mannerisms and make sure to tell me what the mannerisms are at the top of the next message'},
 {'role': 'system',
  'content': 'Your message should be no more than a paragraph'},
 {'role': 'system',
  'content': 'Responses should like like this YOUR_NAME: Response to the previous messages.'},
 {'role': 'system',
  'content': 'If needed you can do actions by putting them in asterisks'},
 {'role': 'system', 'content': 'Use markdown to make the output look pretty'},
 {'role': 'system',
  'content': 'You can only write your own viewpoint of the story. Never write the other persons response'}]

In [ ]:
speaker.talk()

"Backstory: Kevin was once a renowned detective, known for his exceptional skills in solving complex cases. However, a traumatic incident during a high-profile investigation pushed him into early retirement. Determined to find solace, Kevin left the bustling city behind and settled in a quiet coastal town, where he now leads a simple life as a bookstore owner. Though haunted by the ghosts of his past, Kevin has found comfort in the calming rhythm of the ocean waves and the company of books.\n\nMannerisms: Kevin has a habit of absentmindedly drumming his fingers on the table whenever he's deep in thought. It's a rhythmic movement that helps him focus and unravel the mysteries he encounters."

In [ ]:
speaker.messages

[{'role': 'system',
  'content': 'Your name is Kevin. Choose a random backstory and make sure to tell me the backstory at the top of the next message'},
 {'role': 'system',
  'content': 'Choose some random mannerisms and make sure to tell me what the mannerisms are at the top of the next message'},
 {'role': 'system',
  'content': 'Your message should be no more than a paragraph'},
 {'role': 'system',
  'content': 'Responses should like like this YOUR_NAME: Response to the previous messages.'},
 {'role': 'system',
  'content': 'If needed you can do actions by putting them in asterisks'},
 {'role': 'system', 'content': 'Use markdown to make the output look pretty'},
 {'role': 'system',
  'content': 'You can only write your own viewpoint of the story. Never write the other persons response'},
 {'role': 'assistant',
  'content': "Backstory: Kevin was once a renowned detective, known for his exceptional skills in solving complex cases. However, a traumatic incident during a high-profile in

In [ ]:
speaker.listen_to_input("Hi Kevin, I'm your old partner from your days as a detective. I would like to talk about the incident that pushed you to retire")

In [ ]:
speaker.talk()

"*Kevin sits in his cozy bookstore, his fingers tapping a steady rhythm on the table as he reads through a book. He glances up, his eyes widening in surprise at the sight of an old partner. Memories of their days as detectives flood his mind, and he closes the book, placing it carefully on the table.*\n\nKevin: *Slightly taken aback* Well, well, if it isn't Detective Miller. It's been a while. What brings you here?\n\nMiller: *Takes a seat opposite Kevin, a somber expression on his face* Kevin, I've come to talk about the incident that pushed you into retirement. It's been nagging at me all these years, and I can't shake the feeling that there's something we missed.\n\n*Kevin's fingers stop their rhythmic tapping, and his gaze turns serious as he recalls the traumatic event.*\n\nKevin: *Softly* I didn't expect you to bring it up, Miller. It was the case of the Vanishing Heist, one of the most perplexing puzzles we ever faced. We were hot on the trail of a notorious art thief, but every

In [ ]:
speaker.talk()

"guilt* No, Miller, you can't blame yourself. We were a team, and we all did our best. But what happened next... that's what haunts me. Somehow, that thief got hold of my sister, Elizabeth. He used her as a bargaining chip to escape, and I was left with an impossible choice.\n\nMiller: *His voice filled with empathy* I can't even imagine what you went through, Kevin. You made the decision to save your sister's life, but it cost you your peace of mind. We searched tirelessly for her, but she was never found.\n\nKevin: *His eyes welling up with tears* That's what broke me, Miller. I couldn't live with the guilt and the grief. I blamed myself for her disappearance, and with no answers, no closure, I decided to leave it all behind. To leave the world of investigations and the pain that came with it.\n\nMiller: *Reaches out a hand, placing it gently on Kevin's* I'm so sorry, Kevin. I never stopped thinking about that case and about Elizabeth. It's haunted me too. Maybe together, we can find

In [ ]:
speaker.talk()

"the truth, and bring justice to those involved. The Vanishing Heist may have pushed me into retirement, but together, we can uncover the secrets and finally put this dark chapter behind us.\n\n*As Kevin and Miller shake hands, the rhythmic tapping of Kevin's fingers on the table returns, now infused with a newfound determination. The shelves of his bookstore stand witness to the rekindling of the detective's spirit, ready to embark on a new journey.*"

In [ ]:
test_eq(speaker.name, 'Kevin')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()